In [1]:
import sys
import pathlib
import copy

sys.path.append("..")

In [2]:
import mrob 
import numpy as np
import open3d as o3d 

import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import voxel_slam

## I/O

In [4]:
def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [5]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"

In [6]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)

In [7]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=300, number_of_clouds=30, center_distance_threshold=3)

## Optimization

In [8]:
config = voxel_slam.PipelineConfig(voxel_size=2.0, 
                                   ransac_distance_threshold=0.1, 
                                   filter_cosine_distance_threshold=0.2,
                                   backend_verbose=True,
                                   voxel_color_method="voxel")

In [9]:
voxel_filter = voxel_slam.NormalsFilter(config.filter_cosine_distance_threshold)
voxel_filter.set_next(voxel_slam.PlaneDistanceFilter()).set_next(voxel_slam.EmptyVoxelsFilter());

In [10]:
pipeline = voxel_slam.VoxelSLAMPipeline(
    feature_filter=voxel_filter,
    optimization_backend=voxel_slam.BaregBackend,
    config=config
)

In [11]:
pipeline_output = pipeline.process(clouds, poses)

FGraph initial error: 71965.88942832584
Iteratios to converge: 49
Chi2: 55454.78621505488


In [12]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(pipeline_output.optimized_clouds, pipeline_output.optimized_poses)])

## Mini Maps

In [43]:
def get_planes_eigvals(plane_normals): 
    matrix = [] 
    for plane in plane_normals: 
        matrix.append(plane) 
    matrix = np.asarray(matrix) 
    covariance = matrix.T @ matrix 
    eigvals, _ = np.linalg.eig(covariance) 
    return eigvals

In [44]:
def minimap_sampling(clouds, 
                     poses, 
                     config: voxel_slam.PipelineConfig, 
                     optimization_backend, 
                     sample_iterations=30, 
                     sample_ratio=0.7, 
                     plane_df_threshold=1e-1):
    voxel_map = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=config.voxel_size)
    voxel_feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=config.ransac_distance_threshold)

    feature_filter = voxel_slam.NormalsFilter(config.filter_cosine_distance_threshold)
    feature_filter.set_next(voxel_slam.EmptyVoxelsFilter(min_voxel_poses=len(poses)))

    feature_filter.filter(voxel_feature_map)

    feature_map_idx = list(voxel_feature_map.keys())    
    samples_chis = []
    samples_optimized_poses = []
    samples_idx = []
    for _ in range(sample_iterations):
        sampled_voxel_map = {}
        sampled_voxel_idx = random.sample(feature_map_idx, k=int(sample_ratio * len(feature_map_idx)))
        sampled_planes = []

        for voxel_id in sampled_voxel_idx:
            sampled_voxel_map[voxel_id] = copy.deepcopy(voxel_feature_map[voxel_id])
            sampled_planes.append(sampled_voxel_map[voxel_id][0].get_plane_equation()[:-1])
        
        if min(get_planes_eigvals(sampled_planes)) < plane_df_threshold:
            continue
        
        optimized_poses, converged, chi2 = optimization_backend(sampled_voxel_map, len(poses)).get_optimized_poses(number_of_iterations=1000)
        if not converged:
            continue

        samples_chis.append(chi2)
        samples_optimized_poses.append(optimized_poses)
        samples_idx.append(sampled_voxel_idx)

    min_chi_sample_id = np.argmin(samples_chis)
    optimized_poses = samples_optimized_poses[min_chi_sample_id]

    voxel_map_cut = {}
    for k in samples_idx[min_chi_sample_id]:
        voxel_map_cut[k] = copy.deepcopy(voxel_feature_map[k])
        
    optimized_clouds, _ = voxel_map.get_colored_feature_clouds(voxel_map_cut, color_method="voxel")
    
    return optimized_clouds, optimized_poses

In [58]:
def break_on_minimaps(clouds, poses, minimaps_config, aggregate_config, minimap_size=5):
    transformed_clouds = [None for _ in range(len(poses))]
    for i in range(len(poses)):
        transformed_clouds[i] = copy.deepcopy(clouds[i]).transform(poses[i])

    optimized_submaps = []
    for i in range(0, len(poses), minimap_size):
        opt_clouds, opt_poses = minimap_sampling(
            transformed_clouds[i:i+minimap_size],
            [np.eye(4) for _ in range(minimap_size)],
            config=minimaps_config,
            optimization_backend=voxel_slam.BaregBackend,
            plane_df_threshold=15
        )

        optimized_submaps.append(
            voxel_slam.aggregate_map(opt_clouds, opt_poses)
        )

    aggregate_filter = voxel_slam.EmptyVoxelsFilter(min_voxel_poses=2)

    aggregate_pipeline = voxel_slam.VoxelSLAMPipeline(
        feature_filter=aggregate_filter,
        optimization_backend=voxel_slam.BaregBackend,
        config=aggregate_config
    )

    aggregate_output = aggregate_pipeline.process(optimized_submaps, [np.eye(4) for _ in range(len(optimized_submaps))])

    return aggregate_output 

In [59]:
minimaps_config = voxel_slam.PipelineConfig(
    voxel_size=1.0,
    ransac_distance_threshold=0.02,
    filter_cosine_distance_threshold=0.2,
)

aggregate_config = copy.deepcopy(minimaps_config)
aggregate_config.voxel_size = 2.0

In [60]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=300, number_of_clouds=30, center_distance_threshold=3)

In [ ]:
output = break_on_minimaps(clouds, poses, minimaps_config, aggregate_config, minimap_size=5)

In [62]:
o3d.visualization.draw_geometries([
    voxel_slam.aggregate_map(
        output.optimized_clouds,
        output.optimized_poses
    )
])